In [1]:
import requests
import json
from tqdm import tqdm
import time
import os
import pandas as pd
import logging

logging.basicConfig(filename='searched_file_names.log', level=logging.INFO)

In [2]:
api_key = 'Enter Your Neople API-KEY'

In [3]:
def get_creature_data(info):
    serverId = info['server']
    characterId = info['character_ids']
    base_url = f'https://api.neople.co.kr/df/servers/{serverId}/characters/{characterId}/equip/creature?apikey={api_key}'
    
    try:
        res = requests.get(base_url)
    except:
        time.sleep(3)
        res = requests.get(base_url)
    
    if res.status_code == 200:
        return res
    
    return 

In [4]:
def check_searched_job():
    log_file_path = './searched_file_names.log'
    try:
        with open(log_file_path, 'r') as file:
            log_contents = file.read()
            searched_files = log_contents.replace('INFO:root:', '').split('\n')
    except FileNotFoundError:
        searched_files = []
    except Exception as e:
        searched_files = []
    
    return searched_files

In [ ]:
# 아라드 패스1

# user_create_data = pd.read_csv('./data/creature/user_creature_data.csv')


In [ ]:
# # 아라드 패스2
# import pandas as pd
# import os

# df = pd.read_csv('/Users/seokholee/이모저모/dnf/data/creature/user_creature_data_s2.csv')
# path = '/Users/seokholee/이모저모/dnf/data/creature/s2/'


# add_df = pd.concat(
#     [pd.read_csv(path+file_name, encoding='utf-8') for file_name in os.listdir(path)],
# )
# add_df.drop_duplicates().reset_index(drop=True).to_csv('/Users/seokholee/이모저모/dnf/data/creature/user_creature_data_s2.csv')

In [1]:
# # 0425
# import pandas as pd
# import os

# path = '/Users/seokholee/이모저모/dnf/data/creature/s3/'


# add_df = pd.concat(
#     [pd.read_csv(path+file_name, encoding='utf-8') for file_name in os.listdir(path)],
# )
# add_df.drop_duplicates().reset_index(drop=True).to_csv('/Users/seokholee/이모저모/dnf/data/creature/user_creature_data_0425.csv')

In [5]:
path = '/Users/seokholee/이모저모/dnf/data/user_infos/2024PASS_S1/user_info_div/'
file_names = [name for name in os.listdir(path) if '.csv' in name]

for file_name in file_names:
    searched_files = check_searched_job()
    if file_name in searched_files : continue
    user_info = pd.read_csv(path + file_name)
    
    df_data = []
    print('Start GET API : ', file_name)
    for idx, row in tqdm(user_info.iterrows()):
        output = get_creature_data(row)
        if not output : continue
        
        creature_info = output.json()['creature']
        if not creature_info : continue # 크리처가 없는 유저
        
        user_data = [row['server'], row['character_ids']]
        for col in ['itemId', 'itemName', 'itemRarity', 'clone']:
            clone_itemId, clone_itemName = '', ''
            
            if col == 'clone':
                if len(creature_info[col]):
                    user_data.append(creature_info[col]['itemId']), 
                    user_data.append(creature_info[col]['itemName'])
            else:
                user_data.append(creature_info[col])
                
        df_data.append(user_data)
        
    df = pd.DataFrame(df_data,
                      columns= ['serverId','characterId','itemId', 'itemName', 'itemRarity', 'clone_itemId', 'clone_itemName'])
    
    save_path = './data/creature/s3/'
    # user_create_data = pd.concat([user_create_data,df]).drop_duplicates(subset=['serverId', 'characterId']).reset_index(drop=True)
    df.to_csv(save_path+file_name, index=False)
    logging.info(file_name)

Start GET API :  眞 다크 랜서_user_info.csv


1365it [01:00, 24.78it/s]

In [ ]:
def get_item_detail_info(itemId):
    url = f'https://api.neople.co.kr/df/items/{itemId}?apikey={api_key}'
    
    res = requests.get(url)
    
    if res.status_code == 200:
        return res
    
    return 
# res = get_item_detail_info(df.iloc[0]['itemId'])

In [ ]:
df = pd.read_csv('/Users/seokholee/lsh/Project/dnf_creature/dnf_project/data/creature/user_creature_data.csv')

In [ ]:
del_cols = ['itemId','itemRarity', 'itemTypeId', 'itemType',
       'itemTypeDetailId', 'itemTypeDetail', 'itemAvailableLevel',
       'itemExplain', 'itemExplainDetail', 'itemFlavorText', 'obtainInfo',
       'setItemId', 'setItemName', 'itemReinforceSkill', 'creatureInfo']

In [ ]:
creature_infos = []
for name in pd.DataFrame(df['itemName'].value_counts()).index:
    itemId = (df[df['itemName'] == name]['itemId'].iloc[0])
    data = (get_item_detail_info(itemId))
    data = ({key : value for key, value in data.json().items() if key not in del_cols})
    for status in data['itemStatus']:
        data[status['name']] = status['value']
    del data['itemStatus']
    tmp = pd.DataFrame(data, index=[0])
    creature_infos.append(tmp)

In [ ]:
creatures = pd.concat(creature_infos).reset_index(drop=True)
creatures

In [ ]:
creatures.to_csv('./data/creature/active_creature_infos.csv', index=False)

In [ ]:
creature_infos = []
for name in pd.DataFrame(df['clone_itemName'].value_counts()).index:
    itemId = (df[df['clone_itemName'] == name]['clone_itemId'].iloc[0])
    data = (get_item_detail_info(itemId))
    data = ({key : value for key, value in data.json().items() if key in del_cols})
    for status in data['itemStatus']:
        data[status['name']] = status['value']
    del data['itemStatus']
    tmp = pd.DataFrame(data, index=[0])
    creature_infos.append(tmp)

In [ ]:
creatures = pd.concat(creature_infos).reset_index(drop=True)
creatures

In [ ]:
creatures.to_csv('./data/creature/clone_creature_infos.csv', index=False)

In [ ]:
def get_item_sold_price(itemId):
    
    base_url = f'https://api.neople.co.kr/df/auction-sold?itemId={itemId}&wordType=mathch&wordShort=true&limit=100&apikey={api_key}'
    
    res = requests.get(base_url)
    
    if res.status_code == 200:
        return res
    
    return 

In [ ]:
user = pd.read_csv('./data/user_infos_10000.csv')

In [ ]:
user.head()

In [ ]:
def get_character_info(row):
    serverId = row['server']
    character_id = row['character_ids']
    
    base_url = f'https://api.neople.co.kr/df/servers/{serverId}/characters/{character_id}?apikey={api_key}'
    
    try:
        try:
            res = requests.get(base_url)
        except:
            time.sleep(3)
            res = requests.get(base_url)
    except:    
        return 
    
    if res.status_code == 200:
        return res
    
    return 

In [ ]:
adven_list = []
find_fail = 0

In [ ]:
for idx, row in tqdm(user.iterrows()):
    if idx < len(adven_list): continue
    
    
    res = get_character_info(row)
    
    adven_name = res.json()['adventureName'] if res else ''
    
    if adven_name == '' : find_fail+=1
    
    adven_list.append(adven_name)
    

In [ ]:
user['adventureName'] = adven_list
user.head()

In [ ]:
user.tail()

In [ ]:
user.to_csv('./data/user_infos_10000.csv', index=False)

In [ ]:
from datetime import datetime, timedelta

def get_char_history(serverId, characterId, limit = 100, codes = [201,209], search_date = 90):
    """
    Args:
        serverId (_type_): _description_
        characterId (_type_): _description_
        limit (int, optional): _description_. Defaults to 100.
        codes (list, optional): _description_. Defaults to [201,209]. 201 - 레이드, 209 - 레기온
        search_date (int, optional): _description_. Defaults to 90.
        next (str, optional): _description_. Defaults to ''.
    """
    
    
    def get_90_days_ago(today ,search_date):
        return (today - timedelta(days=search_date)).strftime('%Y%m%dT%H%M')


    today = datetime.now()
    endDate = today.strftime('%Y%m%dT%H%M')
    startDate = get_90_days_ago(today, search_date)
    
    for code in codes:
        url = f'https://api.neople.co.kr/df/servers/{serverId}/characters/{characterId}/timeline?limit={limit}&code={code}&startDate={startDate}&endDate={endDate}&apikey={api_key}'
    
        
        res = requests.get(url)
        
        if 'timeline' not in list(res.json().keys()) : continue

        tmp = []
        for row in res.json()['timeline']['rows']:
            data = dict()
            data['server'] = serverId
            data['characterId'] = characterId
            data['code'] = row['code']
            data['date'] = row['date']
            for detail in row['data'].keys():
                data[detail] = row['data'][detail]
            tmp.append(data)
        with open('data/raid_history/history.json', mode='a+', encoding='utf-8') as f:
            json.dump(tmp, f)
            
for idx, row in tqdm(user_info.iterrows()):
    get_char_history(row['server'], row['characterId'])


In [ ]:
user_info = pd.read_csv('data/user_infos/2024PASS_S1/user_infos_10000.csv')